In [299]:
import pandas as pd
import numpy as np
import re

#Webscraping
import requests
from requests import get
from bs4 import BeautifulSoup
import time

In [300]:
df = pd.read_csv('data/michelin_my_maps.csv')

In [301]:
df.head(2)

,Name,Address,Location,Price,Cuisine,Longitude,Latitude,PhoneNumber,Url,WebsiteUrl,Award,FacilitiesAndServices
0,King's Joy,"2 Wudaoying Hutong, Beijing, China Mainland","Beijing, China Mainland",¥¥¥,Vegetarian,116.410004,39.946681,8.610840e+11,https://guide.michelin.com/en/beijing-municipa...,NaN,3 MICHELIN Stars,"Air conditioning,American Express credit card,..."
1,Xin Rong Ji (Xinyuan South Road),"1F, East Tower, Genesis Beijing, 8 Xinyuan Sou...","Beijing, China Mainland",¥¥¥¥,Taizhou,116.450148,39.946380,8.610650e+11,https://guide.michelin.com/en/beijing-municipa...,NaN,3 MICHELIN Stars,"Air conditioning,Car park,China UnionPay,Count..."


In [66]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6780 entries, 0 to 6779
Data columns (total 12 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Name                   6780 non-null   object 
 1   Address                6780 non-null   object 
 2   Location               6780 non-null   object 
 3   Price                  6726 non-null   object 
 4   Cuisine                6780 non-null   object 
 5   Longitude              6780 non-null   float64
 6   Latitude               6780 non-null   float64
 7   PhoneNumber            6649 non-null   float64
 8   Url                    6780 non-null   object 
 9   WebsiteUrl             5542 non-null   object 
 10  Award                  6780 non-null   object 
 11  FacilitiesAndServices  6732 non-null   object 
dtypes: float64(3), object(9)
memory usage: 635.8+ KB


In [263]:
df.Award.value_counts()

Bib Gourmand        3411
1 MICHELIN Star     2744
2 MICHELIN Stars     483
3 MICHELIN Stars     142
Name: Award, dtype: int64

In [264]:
df.Url[2]

'https://guide.michelin.com/en/shanghai-municipality/shanghai/restaurant/taian-table'

In [72]:
#Make all column names lowercase
df.columns = df.columns.str.lower()

In [73]:
#renaming columns in snake case
df = df.rename(columns = {"phonenumber":"phone_number", "websiteurl":"website_url", "facilitiesandservices":"facilities_and_services"})

In [15]:
# parts = url_0.split("en/")
# parts[1]

'beijing-municipality/beijing/restaurant/king-s-joy'

In [16]:
# df = df.assign(parts= df.url.str.split("en/").str[1])

In [302]:
### Using a test df to troubleshoot
df = pd.read_pickle('data/dftest5.pickle')

In [393]:
df.head(2)

,name,address,location,price,cuisine,longitude,latitude,phone_number,url,website_url,award,facilities_and_services,parts,data
0,King's Joy,"2 Wudaoying Hutong, Beijing, China Mainland","Beijing, China Mainland",¥¥¥,Vegetarian,116.410004,39.946681,8.610840e+11,https://guide.michelin.com/en/beijing-municipa...,NaN,3 MICHELIN Stars,"Air conditioning,American Express credit card,...",beijing-municipality/beijing/restaurant/king-s...,
1,Xin Rong Ji (Xinyuan South Road),"1F, East Tower, Genesis Beijing, 8 Xinyuan Sou...","Beijing, China Mainland",¥¥¥¥,Taizhou,116.450148,39.946380,8.610650e+11,https://guide.michelin.com/en/beijing-municipa...,NaN,3 MICHELIN Stars,"Air conditioning,Car park,China UnionPay,Count...",beijing-municipality/beijing/restaurant/xin-ro...,This branch of the chain restaurant opened in ...


In [428]:
#Hardcoding the url for first restaurant
url = 'https://guide.michelin.com/en/beijing-municipality/beijing/restaurant/king-s-joy'

In [313]:
#Checking this to make sure the response is correct
# requests.get(url)
# response = requests.get(url)
# response

<Response [200]>

In [438]:
#Soup factory
request = requests.get(url)
soup = BeautifulSoup(request.content)
more_soup = soup.find('div', class_='restaurant-details__description--text')
page_output = more_soup.find_all('p')

In [439]:
more_soup

<div class="restaurant-details__description--text">
<p></p><p>Just a stone’s throw from Yonghe Temple, this serene restaurant is remodelled from a courtyard house and features a glass-roofed dining room. The chef honed his skills at his grandparents’ restaurant of the same name. Only set menus are available; try the classic menu to sample their specialities all at once. Regulars may opt for the solar-term menu that changes every 15 days. Live harp performance befits the restaurant’s classy elegance.</p>
</div>

In [440]:
page_output

[<p></p>,
 <p>Just a stone’s throw from Yonghe Temple, this serene restaurant is remodelled from a courtyard house and features a glass-roofed dining room. The chef honed his skills at his grandparents’ restaurant of the same name. Only set menus are available; try the classic menu to sample their specialities all at once. Regulars may opt for the solar-term menu that changes every 15 days. Live harp performance befits the restaurant’s classy elegance.</p>]

In [367]:
#telling it you will feed it html
soup = BeautifulSoup(response.content, 'html.parser')

In [270]:
king = soup.find('div', class_='restaurant-details__description--text')
king.find('p', {'class':None} ).text.strip()
#king.find_all('p', class_="{'class':None}")

'Just a stone’s throw from Yonghe Temple, this serene restaurant is remodelled from a courtyard house and features a glass-roofed dining room. The chef honed his skills at his grandparents’ restaurant of the same name. Only set menus are available; try the classic menu to sample their specialities all at once. Regulars may opt for the solar-term menu that changes every 15 days. Live harp performance befits the restaurant’s classy elegance.'

In [171]:
soup.find_all('p', class_='restaurant-details__likeinfo--description pl-text js-favorite__description')

[<p class="restaurant-details__likeinfo--description pl-text js-favorite__description">
             Add to favorites
         </p>,
 <p class="restaurant-details__likeinfo--description pl-text js-favorite__description">
             Add to favorites
         </p>]

In [172]:
soup.find_all('p')

[<p class="restaurant-details__likeinfo--description pl-text js-favorite__description">
             Add to favorites
         </p>,
 <p class="restaurant-details__likeinfo--description pl-text js-favorite__description">
             Add to favorites
         </p>,
 <p><p>Just a stone’s throw from Yonghe Temple, this serene restaurant is remodelled from a courtyard house and features a glass-roofed dining room. The chef honed his skills at his grandparents’ restaurant of the same name. Only set menus are available; try the classic menu to sample their specialities all at once. Regulars may opt for the solar-term menu that changes every 15 days. Live harp performance befits the restaurant’s classy elegance.</p></p>,
 <p>Just a stone’s throw from Yonghe Temple, this serene restaurant is remodelled from a courtyard house and features a glass-roofed dining room. The chef honed his skills at his grandparents’ restaurant of the same name. Only set menus are available; try the classic menu to

In [173]:
soup.select('p:not([class])')

[<p><p>Just a stone’s throw from Yonghe Temple, this serene restaurant is remodelled from a courtyard house and features a glass-roofed dining room. The chef honed his skills at his grandparents’ restaurant of the same name. Only set menus are available; try the classic menu to sample their specialities all at once. Regulars may opt for the solar-term menu that changes every 15 days. Live harp performance befits the restaurant’s classy elegance.</p></p>,
 <p>Just a stone’s throw from Yonghe Temple, this serene restaurant is remodelled from a courtyard house and features a glass-roofed dining room. The chef honed his skills at his grandparents’ restaurant of the same name. Only set menus are available; try the classic menu to sample their specialities all at once. Regulars may opt for the solar-term menu that changes every 15 days. Live harp performance befits the restaurant’s classy elegance.</p>,
 <p>"We offer vegetarian cuisine using green and seasonal produce that is sourced from lo

In [174]:
soup.find('p', {'class':None})

<p><p>Just a stone’s throw from Yonghe Temple, this serene restaurant is remodelled from a courtyard house and features a glass-roofed dining room. The chef honed his skills at his grandparents’ restaurant of the same name. Only set menus are available; try the classic menu to sample their specialities all at once. Regulars may opt for the solar-term menu that changes every 15 days. Live harp performance befits the restaurant’s classy elegance.</p></p>

In [175]:
soup.find('p', {'class':None}).text.strip()

'Just a stone’s throw from Yonghe Temple, this serene restaurant is remodelled from a courtyard house and features a glass-roofed dining room. The chef honed his skills at his grandparents’ restaurant of the same name. Only set menus are available; try the classic menu to sample their specialities all at once. Regulars may opt for the solar-term menu that changes every 15 days. Live harp performance befits the restaurant’s classy elegance.'

In [176]:
soup.find('p', {'class':None}).contents[0]

<p>Just a stone’s throw from Yonghe Temple, this serene restaurant is remodelled from a courtyard house and features a glass-roofed dining room. The chef honed his skills at his grandparents’ restaurant of the same name. Only set menus are available; try the classic menu to sample their specialities all at once. Regulars may opt for the solar-term menu that changes every 15 days. Live harp performance befits the restaurant’s classy elegance.</p>

In [177]:
df['url']

0    https://guide.michelin.com/en/beijing-municipa...
1    https://guide.michelin.com/en/beijing-municipa...
2    https://guide.michelin.com/en/shanghai-municip...
3    https://guide.michelin.com/en/shanghai-municip...
4    https://guide.michelin.com/en/california/san-f...
Name: url, dtype: object

In [311]:
testdf = pd.read_pickle('data/dftest5.pickle')
testdf

,name,address,location,price,cuisine,longitude,latitude,phone_number,url,website_url,award,facilities_and_services,parts
0,King's Joy,"2 Wudaoying Hutong, Beijing, China Mainland","Beijing, China Mainland",¥¥¥,Vegetarian,116.410004,39.946681,8.610840e+11,https://guide.michelin.com/en/beijing-municipa...,NaN,3 MICHELIN Stars,"Air conditioning,American Express credit card,...",beijing-municipality/beijing/restaurant/king-s...
1,Xin Rong Ji (Xinyuan South Road),"1F, East Tower, Genesis Beijing, 8 Xinyuan Sou...","Beijing, China Mainland",¥¥¥¥,Taizhou,116.450148,39.946380,8.610650e+11,https://guide.michelin.com/en/beijing-municipa...,NaN,3 MICHELIN Stars,"Air conditioning,Car park,China UnionPay,Count...",beijing-municipality/beijing/restaurant/xin-ro...
2,Taian Table,"101-102, Building No. 1, Garden Office, No.161...","Shanghai, China Mainland",¥¥¥¥,Innovative,121.474049,31.221807,8.617302e+12,https://guide.michelin.com/en/shanghai-municip...,https://taian-table.cn/,3 MICHELIN Stars,"Air conditioning,American Express credit card,...",shanghai-municipality/shanghai/restaurant/taia...
3,Ultraviolet by Paul Pairet,'somewhere in Shanghai' - meet at Mr & Mrs Bun...,"Shanghai, China Mainland",¥¥¥¥,Innovative,121.485090,31.240358,NaN,https://guide.michelin.com/en/shanghai-municip...,https://uvbypp.cc/,3 MICHELIN Stars,"Air conditioning,American Express credit card,...",shanghai-municipality/shanghai/restaurant/ultr...
4,Quince,"470 Pacific Ave., San Francisco, 94133, USA","San Francisco, USA",$$$$,"Contemporary, Californian",-122.403261,37.797505,1.415776e+10,https://guide.michelin.com/en/california/san-f...,http://www.quincerestaurant.com,3 MICHELIN Stars,"Air conditioning,American Express credit card,...",california/san-francisco/restaurant/quince


In [318]:
df.url[0]

'https://guide.michelin.com/en/beijing-municipality/beijing/restaurant/king-s-joy'

In [319]:
#url of first row
url = 'https://guide.michelin.com/en/beijing-municipality/beijing/restaurant/king-s-joy'

In [320]:
response = get(url)

In [321]:
#response ok
response

<Response [200]>

In [322]:
#Checking this to make sure the response is correct
requests.get(url)

<Response [200]>

In [333]:
print(requests)

<module 'requests' from '/opt/homebrew/anaconda3/lib/python3.9/site-packages/requests/__init__.py'>


In [324]:
# Make a soup variable holding the response content
soup = BeautifulSoup(response.content, 'html.parser')

In [326]:
more_soup = soup.find('div', class_='restaurant-details__description--text')

In [327]:
print(more_soup)

<div class="restaurant-details__description--text">
<p><p>Just a stone’s throw from Yonghe Temple, this serene restaurant is remodelled from a courtyard house and features a glass-roofed dining room. The chef honed his skills at his grandparents’ restaurant of the same name. Only set menus are available; try the classic menu to sample their specialities all at once. Regulars may opt for the solar-term menu that changes every 15 days. Live harp performance befits the restaurant’s classy elegance.</p></p>
</div>


In [328]:
page_output = more_soup.find('p', {'class':None} ).text.strip()

In [329]:
print(page_output)

Just a stone’s throw from Yonghe Temple, this serene restaurant is remodelled from a courtyard house and features a glass-roofed dining room. The chef honed his skills at his grandparents’ restaurant of the same name. Only set menus are available; try the classic menu to sample their specialities all at once. Regulars may opt for the solar-term menu that changes every 15 days. Live harp performance befits the restaurant’s classy elegance.


In [330]:
#Test for first url
output= []
soup = BeautifulSoup(response.content, 'html.parser')
more_soup = soup.find('div', class_='restaurant-details__description--text')
page_output = more_soup.find('p', {'class':None} ).text.strip()
output.append(page_output)
# time.sleep(2)
#df['data'].loc[0] = output

In [331]:
output

['Just a stone’s throw from Yonghe Temple, this serene restaurant is remodelled from a courtyard house and features a glass-roofed dining room. The chef honed his skills at his grandparents’ restaurant of the same name. Only set menus are available; try the classic menu to sample their specialities all at once. Regulars may opt for the solar-term menu that changes every 15 days. Live harp performance befits the restaurant’s classy elegance.']

In [97]:
# def get_michelin_pages():
#     df = pd.read_pickle('data/dftest5.pickle')
#     urls = df['url']
#     output = []
#     michelin_df = pd.DataFrame(columns=['data'])
#     for page in urls:
#         try:
#             soup = BeautifulSoup(requests.get(page).content, 'html.parser')
#             page_output = soup.find('p', {'class':None}).text.strip()
#             output.append(page_output)
#             time.sleep(2)
#             michelin_df['data'].loc[page] = output
#         except:
#             pass
#     michelin_df.to_pickle('data/michelin_df.pickle')

In [101]:
# def get_michelin_pages():
#     df = pd.read_pickle('data/dftest5.pickle')
#     urls = df['url']
#     output = []
#     michelin_df = pd.DataFrame(columns=['data'])
#     for page in urls:
#         response = requests.get(page)
#         soup = BeautifulSoup(response.content, 'html.parser')
#         more_soup = soup.find('div', class_='restaurant-details__description--text')
#         page_output = more_soup.find('p', {'class':None} ).text.strip()
#         output.append(page_output)
#         time.sleep(2)
#         michelin_df['data'].loc[page] = output
#     michelin_df.to_pickle('michelin_df.pickle')

In [105]:
# df = pd.read_pickle('data/dftest5.pickle')

In [256]:
# #Calling test df to test this for problems
# df = pd.read_pickle('data/dftest5.pickle')

In [385]:
#Using 100 to see what we get back
dftest100.head(2)

,name,address,location,price,cuisine,longitude,latitude,phone_number,url,website_url,award,facilities_and_services,parts
0,King's Joy,"2 Wudaoying Hutong, Beijing, China Mainland","Beijing, China Mainland",¥¥¥,Vegetarian,116.410004,39.946681,8.610840e+11,https://guide.michelin.com/en/beijing-municipa...,NaN,3 MICHELIN Stars,"Air conditioning,American Express credit card,...",beijing-municipality/beijing/restaurant/king-s...
1,Xin Rong Ji (Xinyuan South Road),"1F, East Tower, Genesis Beijing, 8 Xinyuan Sou...","Beijing, China Mainland",¥¥¥¥,Taizhou,116.450148,39.946380,8.610650e+11,https://guide.michelin.com/en/beijing-municipa...,NaN,3 MICHELIN Stars,"Air conditioning,Car park,China UnionPay,Count...",beijing-municipality/beijing/restaurant/xin-ro...


In [386]:
#Trying this one, just getting it to append back to the dataframe
def get_michelin_pages(df):
    #df = pd.read_pickle('data/dftest5.pickle')
    urls = df['url']
    output = []
    #michelin_df = pd.DataFrame(columns=['data'])
    for page in urls:
        try:
            request = requests.get(page)
            soup = BeautifulSoup(request.content)
            #more_soup = soup.find('div', class_='restaurant-details__description--text')
            page_output = soup.find('p', {'class':None}).text.strip()
            #page_output = more_soup.find('p', {'class':None}).text.strip()
            output.append(page_output)
            time.sleep(2)
            #michelin_df['data'].loc[page] = output
        except requests.exceptions.RequestException as e:
            print(e)
            continue
    #michelin_df.to_pickle('data/michelin_df.pickle')
    df['data'] = output
    return df

In [387]:
get_michelin_pages(dftest100)

,name,address,location,price,cuisine,longitude,latitude,phone_number,url,website_url,award,facilities_and_services,parts,data
0,King's Joy,"2 Wudaoying Hutong, Beijing, China Mainland","Beijing, China Mainland",¥¥¥,Vegetarian,116.410004,39.946681,8.610840e+11,https://guide.michelin.com/en/beijing-municipa...,NaN,3 MICHELIN Stars,"Air conditioning,American Express credit card,...",beijing-municipality/beijing/restaurant/king-s...,
1,Xin Rong Ji (Xinyuan South Road),"1F, East Tower, Genesis Beijing, 8 Xinyuan Sou...","Beijing, China Mainland",¥¥¥¥,Taizhou,116.450148,39.946380,8.610650e+11,https://guide.michelin.com/en/beijing-municipa...,NaN,3 MICHELIN Stars,"Air conditioning,Car park,China UnionPay,Count...",beijing-municipality/beijing/restaurant/xin-ro...,This branch of the chain restaurant opened in ...
2,Taian Table,"101-102, Building No. 1, Garden Office, No.161...","Shanghai, China Mainland",¥¥¥¥,Innovative,121.474049,31.221807,8.617302e+12,https://guide.michelin.com/en/shanghai-municip...,https://taian-table.cn/,3 MICHELIN Stars,"Air conditioning,American Express credit card,...",shanghai-municipality/shanghai/restaurant/taia...,A fixture on the city’s dining scene since 201...
3,Ultraviolet by Paul Pairet,'somewhere in Shanghai' - meet at Mr & Mrs Bun...,"Shanghai, China Mainland",¥¥¥¥,Innovative,121.485090,31.240358,NaN,https://guide.michelin.com/en/shanghai-municip...,https://uvbypp.cc/,3 MICHELIN Stars,"Air conditioning,American Express credit card,...",shanghai-municipality/shanghai/restaurant/ultr...,This multi-sensory experience at a secret loca...
4,Quince,"470 Pacific Ave., San Francisco, 94133, USA","San Francisco, USA",$$$$,"Contemporary, Californian",-122.403261,37.797505,1.415776e+10,https://guide.michelin.com/en/california/san-f...,http://www.quincerestaurant.com,3 MICHELIN Stars,"Air conditioning,American Express credit card,...",california/san-francisco/restaurant/quince,"An air of refinement infuses this dining room,..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,Le Clos des Sens,"13 rue Jean-Mermoz - à Annecy-le-Vieux, Annecy...","Annecy, France",€€€€,Creative,6.144297,45.918055,3.345023e+10,https://guide.michelin.com/en/auvergne-rhone-a...,https://www.closdessens.com/,3 MICHELIN Stars,"Air conditioning,American Express credit card,...",auvergne-rhone-alpes/annecy/restaurant/le-clos...,This handsome abode in the upper reaches of An...
96,L'Oustau de Baumanière,"Mas de Baumanière, Les Baux-de-Provence, 13520...","Les Baux-de-Provence, France",€€€€,Creative,4.787363,43.740266,3.349054e+10,https://guide.michelin.com/en/provence-alpes-c...,https://www.baumaniere.com/,3 MICHELIN Stars,"Air conditioning,American Express credit card,...",provence-alpes-cote-dazur/les-baux-de-provence...,
97,La Vague d'Or - Cheval Blanc St-Tropez,"Plage de la Bouillabaisse, Saint-Tropez, 83990...","Saint-Tropez, France",€€€€,"Creative, Modern Cuisine",6.626154,43.266585,3.349456e+10,https://guide.michelin.com/en/provence-alpes-c...,https://www.chevalblanc.com/fr/maison/st-tropez/,3 MICHELIN Stars,"Air conditioning,American Express credit card,...",provence-alpes-cote-dazur/saint-tropez/restaur...,
98,Troisgros - Le Bois sans Feuilles,"728 route de Villerest, Ouches, 42155, France","Ouches, France",€€€€,Creative,3.992360,46.010330,3.347772e+10,https://guide.michelin.com/en/auvergne-rhone-a...,https://www.troisgros.com/,3 MICHELIN Stars,"Air conditioning,American Express credit card,...",auvergne-rhone-alpes/ouches/restaurant/troisgr...,


In [391]:
#Number of results missing
len(dftest100[dftest100.data == ''])

41

In [392]:
#save this to dataframe
dftest100.to_pickle('data/test_100_dataframe_41_return')

In [376]:
# request = requests.get(url)
# soup = BeautifulSoup(request.content)
# more_soup = soup.find('div', class_='restaurant-details__description--text')
# page_output = more_soup.find_all('p')[1]

In [103]:
# testdf = pd.read_pickle('data/michelin_df.pickle')

In [99]:
# testdf = pd.read_pickle('data/michelin_df.pickle')

In [374]:
for i in [['abc'], ['def']]

[['abc'], ['def']]

In [ ]:
# df['data'].str.extract(',:}')

In [ ]:
# example_string = re.sub(r'[{}]', '', example_string)

In [46]:
# df.to_pickle('data/testdf.pickle')

In [378]:
dftest = pd.read_pickle('data/testdf.pickle')

In [380]:
dftest.head(2)

,name,address,location,price,cuisine,longitude,latitude,phone_number,url,website_url,award,facilities_and_services,parts
0,King's Joy,"2 Wudaoying Hutong, Beijing, China Mainland","Beijing, China Mainland",¥¥¥,Vegetarian,116.410004,39.946681,8.610840e+11,https://guide.michelin.com/en/beijing-municipa...,NaN,3 MICHELIN Stars,"Air conditioning,American Express credit card,...",beijing-municipality/beijing/restaurant/king-s...
1,Xin Rong Ji (Xinyuan South Road),"1F, East Tower, Genesis Beijing, 8 Xinyuan Sou...","Beijing, China Mainland",¥¥¥¥,Taizhou,116.450148,39.946380,8.610650e+11,https://guide.michelin.com/en/beijing-municipa...,NaN,3 MICHELIN Stars,"Air conditioning,Car park,China UnionPay,Count...",beijing-municipality/beijing/restaurant/xin-ro...


In [381]:
#Make a dataframe of the first 100 to see what we get back
dftest[:100].to_pickle('data/dftest100.pickle')

In [382]:
dftest100 = pd.read_pickle('data/dftest100.pickle')

In [383]:
dftest100

,name,address,location,price,cuisine,longitude,latitude,phone_number,url,website_url,award,facilities_and_services,parts
0,King's Joy,"2 Wudaoying Hutong, Beijing, China Mainland","Beijing, China Mainland",¥¥¥,Vegetarian,116.410004,39.946681,8.610840e+11,https://guide.michelin.com/en/beijing-municipa...,NaN,3 MICHELIN Stars,"Air conditioning,American Express credit card,...",beijing-municipality/beijing/restaurant/king-s...
1,Xin Rong Ji (Xinyuan South Road),"1F, East Tower, Genesis Beijing, 8 Xinyuan Sou...","Beijing, China Mainland",¥¥¥¥,Taizhou,116.450148,39.946380,8.610650e+11,https://guide.michelin.com/en/beijing-municipa...,NaN,3 MICHELIN Stars,"Air conditioning,Car park,China UnionPay,Count...",beijing-municipality/beijing/restaurant/xin-ro...
2,Taian Table,"101-102, Building No. 1, Garden Office, No.161...","Shanghai, China Mainland",¥¥¥¥,Innovative,121.474049,31.221807,8.617302e+12,https://guide.michelin.com/en/shanghai-municip...,https://taian-table.cn/,3 MICHELIN Stars,"Air conditioning,American Express credit card,...",shanghai-municipality/shanghai/restaurant/taia...
3,Ultraviolet by Paul Pairet,'somewhere in Shanghai' - meet at Mr & Mrs Bun...,"Shanghai, China Mainland",¥¥¥¥,Innovative,121.485090,31.240358,NaN,https://guide.michelin.com/en/shanghai-municip...,https://uvbypp.cc/,3 MICHELIN Stars,"Air conditioning,American Express credit card,...",shanghai-municipality/shanghai/restaurant/ultr...
4,Quince,"470 Pacific Ave., San Francisco, 94133, USA","San Francisco, USA",$$$$,"Contemporary, Californian",-122.403261,37.797505,1.415776e+10,https://guide.michelin.com/en/california/san-f...,http://www.quincerestaurant.com,3 MICHELIN Stars,"Air conditioning,American Express credit card,...",california/san-francisco/restaurant/quince
...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,Le Clos des Sens,"13 rue Jean-Mermoz - à Annecy-le-Vieux, Annecy...","Annecy, France",€€€€,Creative,6.144297,45.918055,3.345023e+10,https://guide.michelin.com/en/auvergne-rhone-a...,https://www.closdessens.com/,3 MICHELIN Stars,"Air conditioning,American Express credit card,...",auvergne-rhone-alpes/annecy/restaurant/le-clos...
96,L'Oustau de Baumanière,"Mas de Baumanière, Les Baux-de-Provence, 13520...","Les Baux-de-Provence, France",€€€€,Creative,4.787363,43.740266,3.349054e+10,https://guide.michelin.com/en/provence-alpes-c...,https://www.baumaniere.com/,3 MICHELIN Stars,"Air conditioning,American Express credit card,...",provence-alpes-cote-dazur/les-baux-de-provence...
97,La Vague d'Or - Cheval Blanc St-Tropez,"Plage de la Bouillabaisse, Saint-Tropez, 83990...","Saint-Tropez, France",€€€€,"Creative, Modern Cuisine",6.626154,43.266585,3.349456e+10,https://guide.michelin.com/en/provence-alpes-c...,https://www.chevalblanc.com/fr/maison/st-tropez/,3 MICHELIN Stars,"Air conditioning,American Express credit card,...",provence-alpes-cote-dazur/saint-tropez/restaur...
98,Troisgros - Le Bois sans Feuilles,"728 route de Villerest, Ouches, 42155, France","Ouches, France",€€€€,Creative,3.992360,46.010330,3.347772e+10,https://guide.michelin.com/en/auvergne-rhone-a...,https://www.troisgros.com/,3 MICHELIN Stars,"Air conditioning,American Express credit card,...",auvergne-rhone-alpes/ouches/restaurant/troisgr...


In [421]:
#getting fresh test data
dftest = pd.read_pickle('data/testdf.pickle')
#Make a dataframe of the first 100 to see what we get back
dftest[:100].to_pickle('data/dftest100_iteration2.pickle')
#Reading it back with a new name
dftest100_iteration2 = pd.read_pickle('data/dftest100_iteration2.pickle')

In [441]:
#Testing this function to find out if we can get the remainder
def get_michelin_pages_test2(df):
    #df = pd.read_pickle('data/dftest5.pickle')
    urls = df['url']
    output = []
    #michelin_df = pd.DataFrame(columns=['data'])
    for page in urls:
        try:
            request = requests.get(page)
            soup = BeautifulSoup(request.content)
            more_soup = soup.find('div', class_='restaurant-details__description--text')
            page_output = more_soup.find_all('p', text=True)
            #page_output = more_soup.find('p', {'class':None}).text.strip()
            output.append(page_output)
            time.sleep(2)
            #michelin_df['data'].loc[page] = output
        except requests.exceptions.RequestException as e:
            print(e)
            continue
    #michelin_df.to_pickle('data/michelin_df.pickle')
    df['data'] = output
    return df

In [442]:
get_michelin_pages_test2(dftest100_iteration2)

,name,address,location,price,cuisine,longitude,latitude,phone_number,url,website_url,award,facilities_and_services,parts,data
0,King's Joy,"2 Wudaoying Hutong, Beijing, China Mainland","Beijing, China Mainland",¥¥¥,Vegetarian,116.410004,39.946681,8.610840e+11,https://guide.michelin.com/en/beijing-municipa...,NaN,3 MICHELIN Stars,"Air conditioning,American Express credit card,...",beijing-municipality/beijing/restaurant/king-s...,"[[Just a stone’s throw from Yonghe Temple, thi..."
1,Xin Rong Ji (Xinyuan South Road),"1F, East Tower, Genesis Beijing, 8 Xinyuan Sou...","Beijing, China Mainland",¥¥¥¥,Taizhou,116.450148,39.946380,8.610650e+11,https://guide.michelin.com/en/beijing-municipa...,NaN,3 MICHELIN Stars,"Air conditioning,Car park,China UnionPay,Count...",beijing-municipality/beijing/restaurant/xin-ro...,[[This branch of the chain restaurant opened i...
2,Taian Table,"101-102, Building No. 1, Garden Office, No.161...","Shanghai, China Mainland",¥¥¥¥,Innovative,121.474049,31.221807,8.617302e+12,https://guide.michelin.com/en/shanghai-municip...,https://taian-table.cn/,3 MICHELIN Stars,"Air conditioning,American Express credit card,...",shanghai-municipality/shanghai/restaurant/taia...,[[A fixture on the city’s dining scene since 2...
3,Ultraviolet by Paul Pairet,'somewhere in Shanghai' - meet at Mr & Mrs Bun...,"Shanghai, China Mainland",¥¥¥¥,Innovative,121.485090,31.240358,NaN,https://guide.michelin.com/en/shanghai-municip...,https://uvbypp.cc/,3 MICHELIN Stars,"Air conditioning,American Express credit card,...",shanghai-municipality/shanghai/restaurant/ultr...,[[This multi-sensory experience at a secret lo...
4,Quince,"470 Pacific Ave., San Francisco, 94133, USA","San Francisco, USA",$$$$,"Contemporary, Californian",-122.403261,37.797505,1.415776e+10,https://guide.michelin.com/en/california/san-f...,http://www.quincerestaurant.com,3 MICHELIN Stars,"Air conditioning,American Express credit card,...",california/san-francisco/restaurant/quince,[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,Le Clos des Sens,"13 rue Jean-Mermoz - à Annecy-le-Vieux, Annecy...","Annecy, France",€€€€,Creative,6.144297,45.918055,3.345023e+10,https://guide.michelin.com/en/auvergne-rhone-a...,https://www.closdessens.com/,3 MICHELIN Stars,"Air conditioning,American Express credit card,...",auvergne-rhone-alpes/annecy/restaurant/le-clos...,[[This handsome abode in the upper reaches of ...
96,L'Oustau de Baumanière,"Mas de Baumanière, Les Baux-de-Provence, 13520...","Les Baux-de-Provence, France",€€€€,Creative,4.787363,43.740266,3.349054e+10,https://guide.michelin.com/en/provence-alpes-c...,https://www.baumaniere.com/,3 MICHELIN Stars,"Air conditioning,American Express credit card,...",provence-alpes-cote-dazur/les-baux-de-provence...,[]
97,La Vague d'Or - Cheval Blanc St-Tropez,"Plage de la Bouillabaisse, Saint-Tropez, 83990...","Saint-Tropez, France",€€€€,"Creative, Modern Cuisine",6.626154,43.266585,3.349456e+10,https://guide.michelin.com/en/provence-alpes-c...,https://www.chevalblanc.com/fr/maison/st-tropez/,3 MICHELIN Stars,"Air conditioning,American Express credit card,...",provence-alpes-cote-dazur/saint-tropez/restaur...,[]
98,Troisgros - Le Bois sans Feuilles,"728 route de Villerest, Ouches, 42155, France","Ouches, France",€€€€,Creative,3.992360,46.010330,3.347772e+10,https://guide.michelin.com/en/auvergne-rhone-a...,https://www.troisgros.com/,3 MICHELIN Stars,"Air conditioning,American Express credit card,...",auvergne-rhone-alpes/ouches/restaurant/troisgr...,[[In this nature-inspired interior designed by...


In [449]:
dftest100_iteration2.url[4]

'https://guide.michelin.com/en/california/san-francisco/restaurant/quince'

In [447]:
#Number of results missing
dftest100_iteration2.data == 

0     False
1     False
2     False
3     False
4     False
      ...  
95    False
96    False
97    False
98    False
99    False
Name: data, Length: 100, dtype: bool

In [450]:
url_4 = dftest100_iteration2.url[4]

In [451]:
url_4

'https://guide.michelin.com/en/california/san-francisco/restaurant/quince'

In [466]:
#Trying with url 4 "quince"
request = requests.get(url)
soup = BeautifulSoup(request.content)
more_soup = soup.find('div', class_='restaurant-details__description--text')
page_output = more_soup.select('p:nth-of-type(2)')[0].text

In [467]:
more_soup

<div class="restaurant-details__description--text">
<p></p><p>Just a stone’s throw from Yonghe Temple, this serene restaurant is remodelled from a courtyard house and features a glass-roofed dining room. The chef honed his skills at his grandparents’ restaurant of the same name. Only set menus are available; try the classic menu to sample their specialities all at once. Regulars may opt for the solar-term menu that changes every 15 days. Live harp performance befits the restaurant’s classy elegance.</p>
</div>

In [468]:
page_output

'Just a stone’s throw from Yonghe Temple, this serene restaurant is remodelled from a courtyard house and features a glass-roofed dining room. The chef honed his skills at his grandparents’ restaurant of the same name. Only set menus are available; try the classic menu to sample their specialities all at once. Regulars may opt for the solar-term menu that changes every 15 days. Live harp performance befits the restaurant’s classy elegance.'

In [461]:
#Iteration 3 of function

def get_michelin_pages_test2(df):
    #df = pd.read_pickle('data/dftest5.pickle')
    urls = df['url']
    output = []
    #michelin_df = pd.DataFrame(columns=['data'])
    for page in urls:
        try:
            request = requests.get(page)
            soup = BeautifulSoup(request.content)
            more_soup = soup.find('div', class_='restaurant-details__description--text')
            page_output = more_soup.select('p:nth-of-type(2)')[0].text
            #page_output = more_soup.find('p', {'class':None}).text.strip()
            output.append(page_output)
            time.sleep(2)
            #michelin_df['data'].loc[page] = output
        except requests.exceptions.RequestException as e:
            print(e)
            continue
    #michelin_df.to_pickle('data/michelin_df.pickle')
    df['data'] = output
    return df

In [462]:
get_michelin_pages_test2(dftest100_iteration2)

IndexError: list index out of range